## 이 문서를 수정할 당신에게...
#### 현재 상황은 아래와 같습니다.
1. 우리의 앱에서 중요한 요소 중 하나는 와인을 추천하고, 이를 실제로 구매할 수 있는 링크에 연결하는 것입니다. 또, 실제 와인바를 추천해주기도 합니다. 이를 위해서 실제 와인을 파는 사이트와 와인바를 데이터베이스로 만들어두어야하기 때문에 이를 위해서는 크롤링이 필요합니다.

#### 당신의 목표는 아래와 같습니다.
1. 아래 사이트에 대한 크롤링을 진행합니다. 크롤링이 필요한 요소는 와인바 추천에 필요하다고 생각하는 모든 것을 하면 됩니다. 또한 추가로 더 좋은 사이트의 크롤링도 좋습니다.

    <strong>와인바</strong>
    - 서울내에 있는 와인바(사이트 찾아야함)

2. 와인바에 대한 데이터베이스도 통합합니다. 추가로 다른 schema도 추가해줍니다.
- 상호명
- 위치(위도, 경도)
- 도로명주소
- 메뉴
- 영업시간

#### 참고 사항
현재 작성된 크롤링은 아래의 프레임워크로 작업했습니다.
- [Scrapy](https://docs.scrapy.org/en/latest/intro/tutorial.html)
- [Selenium](https://selenium-python.readthedocs.io/)

In [9]:
import os
from time import sleep
import configparser

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

아래 코드를 실행하면 크롬이 열리고, 와인앤모어 로그인 페이지로 접속하게 된다.

In [16]:
# Setup the driver. This one uses chrome with some options and a path to the chromedriver
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome()

# implicitly_wait tells the driver to wait before throwing an exception
driver.implicitly_wait(30)

# go to the login page
driver.get('https://www.mangoplate.com/search/%EC%99%80%EC%9D%B8?keyword=%EC%99%80%EC%9D%B8&page=1') # 망고플레이트 와인 검색 창으로 이동

In [8]:
driver.get('https://www.mangoplate.com/search/%EC%99%80%EC%9D%B8?keyword=%EC%99%80%EC%9D%B8&page=1')

In [61]:
def get_detail_info(detail_url):
    driver.get(detail_url)
    # Allow time for the website to load
    sleep(10)
    data_dict = {}
    data_dict['url'] = driver.current_url

    name = driver.find_element(By.CSS_SELECTOR, "h1.restaurant_name")
    data_dict['name'] = name.text if name else ''

    rating = driver.find_element(By.CSS_SELECTOR, "strong.rate-point span")
    data_dict['rating'] = rating.text if rating else ''

    try:
        address = driver.find_element(By.XPATH, '//tr[th[contains(text(), "주소")]]/td')
        data_dict['address'] = address.text if address else ''
    except:
        data_dict['address'] = ''

    try:
        phone = driver.find_element(By.XPATH, '//tr[th[contains(text(), "전화번호")]]/td')
        data_dict['phone'] = phone.text if phone else ''
    except:
        data_dict['phone'] = ''

    # 가격대
    try:
        price_range = driver.find_element(By.XPATH, '//tr[th[contains(text(), "가격대")]]/td')
        data_dict['price_range'] = price_range.text if price_range else ''
    except:
        data_dict['price_range'] = ''


    # 주차
    try:
        parking = driver.find_element(By.XPATH, '//tr[th[contains(text(), "주차")]]/td')
        data_dict['parking'] = parking.text if parking else '' 
    except:
        data_dict['parking'] = ''

    # 영업시간
    try:
        opening_hours = driver.find_element(By.XPATH, '//tr[th[contains(text(), "영업시간")]]/td')
        data_dict['opening_hours'] = opening_hours.text if opening_hours else ''
    except:
        data_dict['opening_hours'] = ''

    # 휴일
    try:
        holidays = driver.find_element(By.XPATH, '//tr[th[contains(text(), "휴일")]]/td')
        data_dict['holidays'] = holidays.text if holidays else ''
    except:
        data_dict['holidays'] = ''

    # 메뉴
    menu_items = driver.find_elements(By.XPATH, '//tr[th[contains(text(), "메뉴")]]//li')
    menu = {}
    for item in menu_items:
        menu_name = item.find_element(By.CLASS_NAME, 'Restaurant_Menu').text
        menu_price = item.find_element(By.CLASS_NAME, 'Restaurant_MenuPrice').text
        menu[menu_name] = menu_price
    data_dict['menu'] = menu

    img_elements = driver.find_elements(By.XPATH, '//img[@class="center-croping"]')

    # 각 이미지 요소에서 'src' 속성을 추출합니다.
    img_urls = [img.get_attribute('src') for img in img_elements]
    data_dict['img_urls'] = img_urls

    reviews = driver.find_elements(By.CSS_SELECTOR, 'p.RestaurantReviewItem__ReviewText')
    review = {}
    review_list = []
    for review in reviews:
        review_list.append(review.text)
    data_dict['review'] = review_list   

    return data_dict

In [54]:
get_detail_info('https://www.mangoplate.com/restaurants/Ha-RjMRFLWAZ')

{'url': 'https://www.mangoplate.com/restaurants/Ha-RjMRFLWAZ',
 'name': '미도림',
 'rating': '3.9',
 'address': '서울특별시 성동구 왕십리로 108 3F\n지번 서울시 성동구 성수동1가 656-1069 3F',
 'phone': '02-469-8486',
 'price_range': '만원-2만원',
 'parking': '주차공간없음',
 'opening_hours': '월-금: 16:00 - 24:00\n토-일: 13:00 - 01:00',
 'holidays': '',
 'menu': {'취나물파스타': '14,000원',
  '부추교자': '12,000원',
  '더덕약고추장 곁들인 새우전변': '12,000원',
  '미더덕소스로 버무린 육회': '17,000원',
  '성게파테와 기정떡': '13,000원'},
 'img_urls': ['https://mp-seoul-image-production-s3.mangoplate.com/572183_1624961579224137.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80',
  'https://mp-seoul-image-production-s3.mangoplate.com/576647_1675389463747207.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80',
  'https://mp-seoul-image-production-s3.mangoplate.com/576647_1675389464599858.jpg?fit=around|512:512&crop=512:512;*,*&output-format=jpg&output-quality=80',
  'https://mp-seoul-image-production-s3.mangoplate.com/576647_

아래 코드를 실행하면 페이지를 순회하면 크롤링을 진행하고, 이를 data_list에 저장한다.

In [42]:
# Navigate to product list page
data_list = []
links = set()
for idx in range(1, 11):
    driver.get(f'https://www.mangoplate.com/search/%EC%99%80%EC%9D%B8?keyword=%EC%99%80%EC%9D%B8&page={idx}')
    sleep(10)
    product_links = driver.find_elements(By.CSS_SELECTOR, "a.only-desktop_not")
    for link in product_links:
        links.add(link.get_attribute('href'))
    for i in product_links:
        if i.get_attribute('href') not in links:
            links.add(i.get_attribute('href'))

In [63]:
links_list = list(links)

In [66]:
cnt = 0
for detail_url in tqdm(links):
    cnt += 1
    if cnt <= 3:
        continue
    data_list.append(get_detail_info(detail_url))

100%|██████████| 200/200 [3:13:29<00:00, 58.05s/it]   


In [69]:
import json

In [70]:
with open('./data/wine_bar.json', 'w', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')

In [71]:
driver.close()

### wrtn을 이용한 리뷰 데이터 요약

In [1]:
import pandas as pd

In [2]:
df = pd.read_json('./data/wine_bar.json', encoding='utf-8', lines=True)

In [56]:
driver.quit()

In [4]:
for idx in range(len(df['review'])):
    df['review'][idx] = df['review'][idx].replace('\n', '')

C:\Users\chois\AppData\Local\Temp\ipykernel_14736\4108177843.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'][idx] = df['review'][idx].replace('\n', '')


In [5]:
import time

In [10]:
# Setup the driver. This one uses chrome with some options and a path to the chromedriver
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome()

# implicitly_wait tells the driver to wait before throwing an exception
driver.implicitly_wait(30)

# # go to the login page
# driver.get('https://wrtn.ai/') 

In [51]:

for idx in range(161, 162):
    driver.get('https://wrtn.ai/') 
    sleep(10)
    input_textarea = driver.find_element(By.CSS_SELECTOR, "div.sc-gsnTZi > textarea")

    llm_model_buttons = driver.find_elements(By.CSS_SELECTOR, "div.sc-hFvvYD")

    for llm_model_button in llm_model_buttons:
        if llm_model_button.text == 'GPT-4':
            break  # 원하는 요소를 찾았으므로 반복문을 종료합니다.

    submit_buttons = driver.find_element(By.CSS_SELECTOR, "div.sc-gsnTZi.sc-lkcIho")
    llm_model_button.click()

    prompt = f"""이 리뷰를 보고, 가게 정보를 요약해줘: {df['review'][idx]}"""
    input_textarea.send_keys(prompt)
    
    submit_buttons.click()
    sleep(60)
    review_elements = driver.find_elements(By.CSS_SELECTOR, "div.sc-gauFBm.ecjxKC>p")
    review_text = ''
    for i in range(len(review_elements)):
        review_text += review_elements[i].text
    review_text= review_text.replace(prompt, '')
    df['summary'][idx] = review_text
    print(df['name'][idx], df['summary'][idx])


르챔버 이 리뷰를 보고, 가게 정보를 요약해줘: ['너무 유명한 곳이지만, 이름값 못한다고 느꼈던 곳\n\n커버차지: 만원\n기본안주: 생수/탄산수 택 1, 과자\n추천메뉴: 챔버스토리\n\n한국 스피크이지바 열풍의 역사를 시작한 곳이자 내로라하는 많은 바텐더분들이 한번씩 거쳐갔다는 르챔버. 정말 손님이 많다. 오픈시간 맞춰가니 입장이 가능했지만 곧 만석이었고 9시경 나올 때는 웨이팅이 길었다. 외국인 손님들도 많이 보인다.\n\n전반적으로 시그니처 칵테일들이 78년 전에는 참신했을지 모르나 이제는 특별함이 많이 부족한 느낌이었고 비주얼이 맛보다 강하다는 느낌을 받았다. 관광객, 인스타그래머들을 겨냥한 건지 달고 대중적인데다 맛이 입 안에서 변하는 색다른 재미나 깊이가 떨어지는 느낌. 그렇다고 공간과 서비스가 압도적이냐 하면 그것도 아니다. 캐주얼한 바라고 하기에는 가격대가 잔당 270003만원 선에 커버차지도 받으니 사람 씀씀이는 다 다르지만 나한테는 캐주얼한 가격대가 아니고.... 그렇다고 공간이 훌륭하거나 접객이 세심하거나 기본안주 구성이 좋은 것도 아니고 좀 많이 애매한 분위기. 홀 좌석에 등받이 없는 의자로 안내해주셔서 의자를 청하니 바틀로 주문하면 등받이 있는 자리로 옮겨준다는 말까지 좀 어이없었다 ㅋㅋㅋ\n\n첫잔 Chamber story\nDark rum, pinepple, lime\nSweet vermouth, bitter, hazelnut\n\n훈연향이 은근히 풍기고 럼 맛은 약한 편. 상자가 스르륵 열리며 연기가 뿜어나오고 보석처럼 빛나는 칵테일잔은 확실히 엄청 인스타그래머블하다. 괜히 수많은 업장에서 따라하는 게 아니다. 정말 예뻤다. 비주얼에 미치지 못했을 뿐 맛도 무난하게 맛있다.\n\n2. Laughing buddha\nLychee, pineapple, lime, butter, yeontae kaliang\n\n굉장히 고급진 열대과일주스 맛~ 고량주 향이 뒤에 풍기면서 약간의 킥을 선사해서 물리지 않게 마실 수 있다. 부처상 모양의 

C:\Users\chois\AppData\Local\Temp\ipykernel_14736\3549620362.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['summary'][idx] = review_text


In [54]:
import json

In [55]:
# convert DataFrame to list of dicts
data_list = df.to_dict(orient='records')

# write to file
with open('./data/wine_bar.json', 'w', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')